# Importando as bibliotecas

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

%matplotlib inline

# Base Info:
Nome: Rice Dataset (Commeo and Osmancik)     
Resumo: Um total de 3810 imagens de grãos de arroz foram tiradas de duas espécies (Cammeo e Osmancik).      
- As imagens foram pré-processadas e certos atributos foram inferidos     
- Sete atributos morfológicos foram obtidos para cada imagem de grão de arroz
    - Area (integer)
    - Perimeter (float)
    - **Major_Axis_Length (float)** -> atributos utilizados
    - **Minor_Axis_Length (float)** -> atributos utilizados
    - Eccentricity (float)
    - Extent (float)
- Classes:
    - Cammeo
    - Osmancik

	



In [43]:
df = pd.read_csv('rice.csv', header=None)
y = df.iloc[0:, 7].values
y.size
df.head

<bound method NDFrame.head of           0           1           2          3         4      5         6  \
0     15231  525.578979  229.749878  85.093788  0.928882  15617  0.572896   
1     14656  494.311005  206.020065  91.730972  0.895405  15072  0.615436   
2     14634  501.122009  214.106781  87.768288  0.912118  14954  0.693259   
3     13176  458.342987  193.337387  87.448395  0.891861  13368  0.640669   
4     14688  507.166992  211.743378  89.312454  0.906691  15262  0.646024   
...     ...         ...         ...        ...       ...    ...       ...   
3805  11441  415.858002  170.486771  85.756592  0.864280  11628  0.681012   
3806  11625  421.390015  167.714798  89.462570  0.845850  11904  0.694279   
3807  12437  442.498993  183.572922  86.801979  0.881144  12645  0.626739   
3808   9882  392.296997  161.193985  78.210480  0.874406  10097  0.659064   
3809  11434  404.709991  161.079269  90.868195  0.825692  11591  0.802949   

             7  
0       Cammeo  
1       Cam

In [44]:
y = np.where(y == 'Cammeo', -1, 1) # realizado o tratamento do classe categorica (string -> int)
y

array([-1, -1, -1, ...,  1,  1,  1])

In [51]:
# X = df.iloc[0:, [2,3]].values # atributos utilizados
# X.size

X = df.iloc[0:, [1,2,3]].values # atributos utilizados
print(type(X.dtype))
# X
# X = X.round(decimals=3)
# X

<class 'numpy.dtypes.Float64DType'>


# Separando a base em treinamento e teste

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=102) # 80% treinamento 20% teste

# Objeto Neuronio, seus atributos e métodos

In [53]:
# base
# classe Neuronio e aprendizado estocastico/online

class Neuronio(object):

    def __init__(self, eta=0.1, epocas=50, fativ='perceptron',showErro=False):
        self.eta = eta
        self.epocas = epocas
        self.w_ = np.random.rand(1 + X.shape[1]) - 0.5
        self.fativ = fativ
        self.showErro = showErro
        self.erro_ep = 0
        self.SqError_ = []

    def somat(self, X):
        return self.w_[0] + np.dot(X, self.w_[1:])

    def sinal(self, x):
        return np.where(x >= 0.0, 1, -1)

    def tanh(self, x):
      return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def d_tanh(self, x):
      return 1 - self.tanh(x) * self.tanh(x)

    def ReLU(self, x):
      return np.where(x > 0.0, x, 0.1*x)  #max(0, x) .... float

    def d_ReLU(self, x):
      return np.where(x > 0.0, 1, 0.1)

    def predict(self,X):
      if self.fativ == 'sinal' or self.fativ == 'perceptron':
#        print(f"sinalPredict={self.sinal(self.somat(X))}")  # debug
        return self.sinal(self.somat(X))
      elif self.fativ == 'linear':
        return self.somat(X)
      elif self.fativ == 'tanh':
        return self.tanh(self.somat(X))
      elif self.fativ == 'ReLU':
        return self.ReLU(self.somat(X))
      else:
        return 11

    def deltaW(self, erro):
      if self.fativ == 'sinal' or self.fativ == 'perceptron':
        atualiza = self.eta * erro
      elif self.fativ == 'linear':
        atualiza = self.eta * erro
      elif self.fativ == 'tanh':
#        print(f"> eta={self.eta} tanh={self.tanh(erro)} dtanh={self.d_tanh(erro)} erro={erro}")
        atualiza = self.eta * self.d_tanh(erro) * erro
      elif self.fativ == 'ReLU':
        atualiza = self.eta * self.d_ReLU(erro) * erro
      else:
        print(f"Função de ativação '{self.fativ}' desconhecida")
        exit()
      return atualiza

    def treinaGD(self,X,y):
      self.SqError_ = []
      self.erros_classif_ = []
      for ep in range(self.epocas):
        erro_classif = 0
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        Xs = X[indices]
        ys = y[indices]
        outputs = self.predict(Xs)
        erros = (ys - outputs)
       # print(outputs,erros)
        erro_classif = np.where(ys*outputs < 0, 1, 0).sum()
        self.erros_classif_.append(erro_classif)
        if ep < (self.epocas-1):
          self.w_[1:] += self.eta * Xs.T.dot(erros)
          self.w_[0] += self.eta * erros.sum()
        SqError = (erros**2).sum() / 2.0 # ou np.square(erros)/2
        self.SqError_.append(SqError)
        if self.showErro:
          if ep == 0:
            print(f"{'Época':^10}\tErro")
          else:
            print(f"{str(ep):^10}\t{erro_classif}")
      return self

    def treinaSGD(self, X, y):
        self.erros_ = []
        self.erros_classif_ = []
        self.SqError_ = []
        for ep in range(self.epocas):
            erro_ep = 0
            erro_classif = 0
            SqError = 0

            indices = np.arange(X.shape[0])
            np.random.shuffle(indices)
            Xs = X[indices]
            ys = y[indices]

            for xi, target in zip(Xs, ys):
              output = self.predict(xi)
              erro = target - output
              SqError += erro*erro
              erro_ep += erro
              if (float(target)*float(output)) < 0:
                #print(f"ep={ep} {xi} target={target} output={output} t*o={target*output}") # debug
                erro_classif += 1

              if (ep < (self.epocas-1)):  #
                atualizacao = self.deltaW(erro)
                self.w_[0] +=  atualizacao * 1
                self.w_[1:] +=  atualizacao * xi

            self.erros_.append(erro_ep)
            self.SqError_.append(SqError)
            self.erros_classif_.append(erro_classif)
            if self.showErro:
              if ep == 0:
                print(f"{'Época':^10}\tErro")
              else:
                print(f"{str(ep):^10}\t{erro_classif}")

        return self

In [54]:
classifier_ = []
W0_ = []

np.random.seed(35)
classifier = Neuronio(fativ='tanh', eta=0.0001, epocas=300, showErro=True)
W0 = classifier.w_.copy()
W0_.append(W0)
classifier.treinaSGD(X_train, y_train)

  Época   	Erro
    1     	1341
    2     	1290
    3     	1159
    4     	1121
    5     	1102
    6     	1037
    7     	988
    8     	998
    9     	948
    10    	942
    11    	959
    12    	905
    13    	867
    14    	903
    15    	906
    16    	876
    17    	886
    18    	912
    19    	895
    20    	886
    21    	888
    22    	879
    23    	878
    24    	887
    25    	865
    26    	864
    27    	888
    28    	848
    29    	841
    30    	875
    31    	836
    32    	876
    33    	831
    34    	842
    35    	820
    36    	830
    37    	852
    38    	843
    39    	834
    40    	824
    41    	838
    42    	796
    43    	843
    44    	822
    45    	816
    46    	867
    47    	823
    48    	823
    49    	851
    50    	838
    51    	831
    52    	813
    53    	822
    54    	859
    55    	848
    56    	777
    57    	803
    58    	837
    59    	830
    60    	813
    61    	810
    62    	819
    63    	790
    64    	824
    65    	817
   

# Métricas

In [55]:
pred = classifier.predict(X_test)
PercClassifica = (100 * np.where(pred*y_test < 0, 0, 1).sum() / len(y_test))
PercClassifica


78.08398950131233